In [ ]:
import os
import random

FloatObject (b'0.00-70') invalid; use 0.0 instead
FloatObject (b'0.00-70') invalid; use 0.0 instead
FloatObject (b'0.00-70') invalid; use 0.0 instead
FloatObject (b'0.00-70') invalid; use 0.0 instead
FloatObject (b'0.00-70') invalid; use 0.0 instead
FloatObject (b'0.00-70') invalid; use 0.0 instead
FloatObject (b'0.00-70') invalid; use 0.0 instead
FloatObject (b'0.00-70') invalid; use 0.0 instead
FloatObject (b'0.00-70') invalid; use 0.0 instead
FloatObject (b'0.000000000-19915751') invalid; use 0.0 instead
FloatObject (b'0.000000000-23235042') invalid; use 0.0 instead
FloatObject (b'0.0000000000-33192919') invalid; use 0.0 instead
FloatObject (b'0.0000000000-33192919') invalid; use 0.0 instead
FloatObject (b'0.000000000-19915751') invalid; use 0.0 instead
FloatObject (b'0.000000000-23235042') invalid; use 0.0 instead
FloatObject (b'0.0000000000-33192919') invalid; use 0.0 instead
FloatObject (b'0.000000000-19915751') invalid; use 0.0 instead
FloatObject (b'0.000000000-23235042') inval

: 

: 

: 

In [1]:
print(f"Train files: {len(train_files)}")

NameError: name 'train_files' is not defined

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig

# Quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load model with quantization
model_path = "./Llama3.2-3B"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    device_map="auto"
)

# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none"
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.gradient_checkpointing_enable()  # Enable gradient checkpointing

In [ ]:
from transformers import TrainingArguments, Trainer
from datasets import load_dataset

# Load dataset
train_dataset = load_dataset('text', data_dir='train_data', split='train')
test_dataset = load_dataset('text', data_dir='test_data', split='test')

training_args = TrainingArguments(
    output_dir="./llama-climate",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    fp16=True,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.01
)

# Data collator
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
import math

def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    return math.exp(loss.item())

# Evaluate on test set
test_texts = [open(f"test_data/{f}.txt").read() for f in test_files]
perplexities = [calculate_perplexity(model, tokenizer, text) for text in test_texts]
print(f"Average Perplexity: {sum(perplexities)/len(perplexities):.2f}")